# 1. Import libraries and load the dataset

In [1]:
!pip install attention

In [2]:
import numpy as np
from numpy import newaxis
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout, Activation
from keras import optimizers

from attention import Attention
from keras.models import Sequential
from keras import optimizers

In [3]:
df = pd.read_csv("./USDT.csv")
df1 = df.reset_index()['Price']
df1.head()

0    1.0100
1    0.9959
2    0.9977
3    1.0000
4    1.0000
Name: Price, dtype: float64

Data preprocessing

In [4]:
# 3. Scaler data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

# 2. Split the data into training, test and validate  sets

In [5]:
train_size = int(0.7 * len(df1))
test_size = int(0.2 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1362, 1)
Test shape: (389, 1)
Validate shape: (196, 1)


In [6]:
train_size = int(0.6 * len(df1))
test_size = int(0.2 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1168, 1)
Test shape: (389, 1)
Validate shape: (390, 1)


In [7]:
train_size = int(0.7 * len(df1))
test_size = int(0.15 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Validate shape:", val_data.shape)

Train shape: (1362, 1)
Test shape: (292, 1)
Validate shape: (293, 1)


In [8]:
# 5. Hàm Create Dataset
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [9]:
#6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_val, y_val = create_dataset(val_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)
print(len(X_test))

191


In [10]:
# 7. Reshape input to be [samples, time steps, features] which is required for LSTM

X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

# 3. Perform RNN-Attention Model

## 3.1 Build the RNN Attention Model

In [11]:
model = Sequential([
    SimpleRNN(100, return_sequences=True, input_shape=(time_step, 1)),
    Attention(),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 100, 100)          10200     
                                                                 
 attention (Attention)       (None, 128)               35600     
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 49,961
Trainable params: 49,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=300,batch_size=64,verbose=1)

Epoch 1/300
20/20 [==============================] - 18s 193ms/step - loss: 0.0233 - accuracy: 0.0000e+00 - val_loss: 9.6601e-04 - val_accuracy: 0.0000e+00
Epoch 2/300
20/20 [==============================] - 1s 69ms/step - loss: 0.0040 - accuracy: 0.0000e+00 - val_loss: 6.1527e-04 - val_accuracy: 0.0000e+00
Epoch 3/300
20/20 [==============================] - 1s 70ms/step - loss: 0.0023 - accuracy: 0.0000e+00 - val_loss: 3.6091e-04 - val_accuracy: 0.0000e+00
Epoch 4/300
20/20 [==============================] - 1s 71ms/step - loss: 0.0019 - accuracy: 7.9302e-04 - val_loss: 9.7417e-04 - val_accuracy: 0.0000e+00
Epoch 5/300
20/20 [==============================] - 1s 71ms/step - loss: 0.0017 - accuracy: 7.9302e-04 - val_loss: 1.0991e-04 - val_accuracy: 0.0000e+00
Epoch 6/300
20/20 [==============================] - 1s 74ms/step - loss: 0.0015 - accuracy: 7.9302e-04 - val_loss: 7.3561e-05 - val_accuracy: 0.0000e+00
Epoch 7/300
20/20 [==============================] - 1s 66ms/step - loss: 

Epoch 54/300
20/20 [==============================] - 2s 78ms/step - loss: 9.2299e-04 - accuracy: 7.9302e-04 - val_loss: 1.8753e-04 - val_accuracy: 0.0000e+00
Epoch 55/300
20/20 [==============================] - 1s 74ms/step - loss: 0.0011 - accuracy: 7.9302e-04 - val_loss: 8.5119e-04 - val_accuracy: 0.0000e+00
Epoch 56/300
20/20 [==============================] - 1s 75ms/step - loss: 0.0014 - accuracy: 7.9302e-04 - val_loss: 6.0285e-04 - val_accuracy: 0.0000e+00
Epoch 57/300
20/20 [==============================] - 2s 81ms/step - loss: 9.9316e-04 - accuracy: 7.9302e-04 - val_loss: 4.1077e-05 - val_accuracy: 0.0000e+00
Epoch 58/300
20/20 [==============================] - 2s 85ms/step - loss: 8.2533e-04 - accuracy: 7.9302e-04 - val_loss: 5.5697e-05 - val_accuracy: 0.0000e+00
Epoch 59/300
20/20 [==============================] - 2s 83ms/step - loss: 8.0667e-04 - accuracy: 7.9302e-04 - val_loss: 2.4085e-05 - val_accuracy: 0.0000e+00
Epoch 60/300
20/20 [==============================] - 

20/20 [==============================] - 1s 56ms/step - loss: 7.5632e-04 - accuracy: 7.9302e-04 - val_loss: 2.9178e-05 - val_accuracy: 0.0000e+00
Epoch 106/300
20/20 [==============================] - 1s 57ms/step - loss: 6.5809e-04 - accuracy: 7.9302e-04 - val_loss: 8.3511e-05 - val_accuracy: 0.0000e+00
Epoch 107/300
20/20 [==============================] - 1s 55ms/step - loss: 7.3553e-04 - accuracy: 7.9302e-04 - val_loss: 4.4588e-05 - val_accuracy: 0.0000e+00
Epoch 108/300
20/20 [==============================] - 1s 59ms/step - loss: 6.8901e-04 - accuracy: 7.9302e-04 - val_loss: 4.1450e-05 - val_accuracy: 0.0000e+00
Epoch 109/300
20/20 [==============================] - 1s 60ms/step - loss: 6.2751e-04 - accuracy: 7.9302e-04 - val_loss: 2.9718e-05 - val_accuracy: 0.0000e+00
Epoch 110/300
20/20 [==============================] - 1s 57ms/step - loss: 7.0978e-04 - accuracy: 7.9302e-04 - val_loss: 4.1832e-04 - val_accuracy: 0.0000e+00
Epoch 111/300
20/20 [==============================] -

20/20 [==============================] - 1s 63ms/step - loss: 5.2519e-04 - accuracy: 7.9302e-04 - val_loss: 6.4751e-05 - val_accuracy: 0.0000e+00
Epoch 157/300
20/20 [==============================] - 1s 58ms/step - loss: 5.9486e-04 - accuracy: 7.9302e-04 - val_loss: 2.5826e-05 - val_accuracy: 0.0000e+00
Epoch 158/300
20/20 [==============================] - 1s 53ms/step - loss: 5.3040e-04 - accuracy: 7.9302e-04 - val_loss: 2.3290e-05 - val_accuracy: 0.0000e+00
Epoch 159/300
20/20 [==============================] - 1s 53ms/step - loss: 5.8320e-04 - accuracy: 7.9302e-04 - val_loss: 5.2571e-04 - val_accuracy: 0.0000e+00
Epoch 160/300
20/20 [==============================] - 1s 62ms/step - loss: 6.5913e-04 - accuracy: 7.9302e-04 - val_loss: 1.8999e-05 - val_accuracy: 0.0000e+00
Epoch 161/300
20/20 [==============================] - 1s 66ms/step - loss: 5.5879e-04 - accuracy: 7.9302e-04 - val_loss: 2.0762e-04 - val_accuracy: 0.0000e+00
Epoch 162/300
20/20 [==============================] -

20/20 [==============================] - 1s 64ms/step - loss: 5.3675e-04 - accuracy: 7.9302e-04 - val_loss: 1.1856e-04 - val_accuracy: 0.0000e+00
Epoch 208/300
20/20 [==============================] - 1s 68ms/step - loss: 5.6333e-04 - accuracy: 7.9302e-04 - val_loss: 2.4261e-05 - val_accuracy: 0.0000e+00
Epoch 209/300
20/20 [==============================] - 1s 67ms/step - loss: 5.0475e-04 - accuracy: 7.9302e-04 - val_loss: 3.5719e-04 - val_accuracy: 0.0000e+00
Epoch 210/300
20/20 [==============================] - 1s 66ms/step - loss: 5.1745e-04 - accuracy: 7.9302e-04 - val_loss: 2.9803e-04 - val_accuracy: 0.0000e+00
Epoch 211/300
20/20 [==============================] - 1s 62ms/step - loss: 4.8179e-04 - accuracy: 7.9302e-04 - val_loss: 2.9474e-05 - val_accuracy: 0.0000e+00
Epoch 212/300
20/20 [==============================] - 1s 66ms/step - loss: 4.5393e-04 - accuracy: 7.9302e-04 - val_loss: 2.2161e-05 - val_accuracy: 0.0000e+00
Epoch 213/300
20/20 [==============================] -

20/20 [==============================] - 1s 52ms/step - loss: 4.8835e-04 - accuracy: 7.9302e-04 - val_loss: 1.8648e-05 - val_accuracy: 0.0000e+00
Epoch 259/300
20/20 [==============================] - 1s 48ms/step - loss: 4.7480e-04 - accuracy: 7.9302e-04 - val_loss: 1.7724e-04 - val_accuracy: 0.0000e+00
Epoch 260/300
20/20 [==============================] - 1s 46ms/step - loss: 4.5516e-04 - accuracy: 7.9302e-04 - val_loss: 7.1526e-05 - val_accuracy: 0.0000e+00
Epoch 261/300
20/20 [==============================] - 1s 45ms/step - loss: 4.6204e-04 - accuracy: 7.9302e-04 - val_loss: 2.3286e-04 - val_accuracy: 0.0000e+00
Epoch 262/300
20/20 [==============================] - 1s 47ms/step - loss: 4.6856e-04 - accuracy: 7.9302e-04 - val_loss: 1.3325e-04 - val_accuracy: 0.0000e+00
Epoch 263/300
20/20 [==============================] - 1s 47ms/step - loss: 4.1440e-04 - accuracy: 7.9302e-04 - val_loss: 3.3856e-05 - val_accuracy: 0.0000e+00
Epoch 264/300
19/20 [===========================>..] -

In [ ]:
def evaluate_rnn_attention_model(actual, X_data):
    y_pred = model.predict(X_data)
    mse = mean_squared_error(actual, y_pred)
    mae = mean_absolute_error(actual, y_pred)
    mape = mean_absolute_percentage_error(actual, y_pred)
    rmse = mean_squared_error(actual, y_pred, squared=False)
    return y_pred, mse, mae, mape, rmse

## 3.2 Evaluate on Validate data

In [ ]:
y_pred_val, val_mse, val_mae, val_mape, val_rmse = evaluate_rnn_attention_model(y_val, X_val)

## 3.3 Evaluate on Test data

In [ ]:
y_pred_test, test_mse, test_mae, test_mape, test_rmse = evaluate_rnn_attention_model(y_test, X_test)

In [ ]:
train_predict = model.predict(X_train)
train_predict = scaler.inverse_transform(train_predict)
y_pred_test = scaler.inverse_transform(y_pred_test)
y_pred_val = scaler.inverse_transform(y_pred_val)

In [ ]:
# 13. Dự báo 30 ngày tiếp theo
x_input=val_data[60:].reshape(1,-1)
x_input.shape

temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=135
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input= x_input.reshape(1,-1)
        print("x_input:",x_input)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1


In [ ]:
metrics = {
    'MSE': [val_mse, test_mse],
    'MAE': [val_mae, test_mae],
    'MAPE': [val_mape, test_mape],
    'RMSE': [val_rmse, test_rmse],
}

acc = pd.DataFrame(metrics, index=['Validate', 'Test'])

print(acc)

In [ ]:
# 14.Vẽ hình

train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))

test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred_test))

val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)

prediect_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))

plt.legend(['Train','Test','Predict','Validate','ValidatePred','Predict30days'])
plt.show()